In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
import nibabel as nib
import cv2

In [3]:
# Load your preprocessed data
data_path = '/content/drive/MyDrive/ACDC/database/training'

In [4]:
list_data = os.listdir(data_path)

In [5]:
len(list_data)

102

In [6]:
samples = list_data
samples

['patient007',
 'patient001',
 'patient003',
 'patient005',
 'patient006',
 'patient004',
 'patient002',
 'patient008',
 'patient009',
 'patient010',
 'patient015',
 'patient012',
 'patient017',
 'patient019',
 'patient018',
 'patient013',
 'patient016',
 'patient011',
 'patient014',
 'patient020',
 'patient024',
 'patient025',
 'patient028',
 'patient023',
 'patient026',
 'patient021',
 'patient027',
 'patient022',
 'patient029',
 'patient030',
 'patient031',
 'patient038',
 'patient036',
 'patient039',
 'patient033',
 'patient034',
 'patient037',
 'patient035',
 'patient032',
 'patient040',
 'patient041',
 'patient042',
 'patient043',
 'patient044',
 'patient046',
 'patient045',
 'patient051',
 'patient047',
 'patient048',
 'patient050',
 'patient049',
 'patient052',
 'patient053',
 'patient054',
 'patient061',
 'patient062',
 'patient063',
 'patient057',
 'patient056',
 'patient058',
 'patient060',
 'patient055',
 'patient059',
 'patient064',
 'patient069',
 'patient067',
 'patient0

In [7]:
new_data = {
    'x':[],
    'y':[]
}
for sample in samples:
  try:
    folder_path = os.path.join(data_path, sample)
    list_file = os.listdir(folder_path)
    print(list_file)
    # load classification groundtruth
    with open(os.path.join(folder_path, 'Info.cfg'), 'r') as f:
      value = f.read().split('\n')
      value = {v.split(':')[0]: v.split(':')[1].strip() for v in value if ':' in v}
      group = value['Group']

    file_to_be_loaded = [l for l in list_file if 'gt' in l][0]
    print(file_to_be_loaded)
    # load classification input
    nii_img  = nib.load(os.path.join(folder_path, file_to_be_loaded))
    nii_data = nii_img.get_fdata()
    if nii_data.shape[-1]<18:
      nii_data_temp = np.zeros(nii_data.shape[:2]+(18,))
      nii_data_temp[:,:,:nii_data.shape[-1]] = nii_data
      nii_data = nii_data_temp

    new_nii_data = cv2.resize(nii_data, (256,256))
    print(new_nii_data.shape)

    new_data['x'].append(new_nii_data)
    new_data['y'].append(group)
  except:
    print('ignore')

['patient007_4d.nii.gz', 'patient007_frame07.nii.gz', 'patient007_frame07_gt.nii.gz', 'Info.cfg', 'patient007_frame01.nii.gz', 'patient007_frame01_gt.nii.gz']
patient007_frame07_gt.nii.gz
(256, 256, 18)
['patient001_frame01.nii.gz', 'patient001_4d.nii.gz', 'patient001_frame12_gt.nii.gz', 'Info.cfg', 'patient001_frame01_gt.nii.gz', 'patient001_frame12.nii.gz']
patient001_frame12_gt.nii.gz
(256, 256, 18)
['patient003_frame01.nii.gz', 'patient003_frame01_gt.nii.gz', 'patient003_frame15.nii.gz', 'patient003_frame15_gt.nii.gz', 'patient003_4d.nii.gz', 'Info.cfg']
patient003_frame01_gt.nii.gz
(256, 256, 18)
['patient005_frame13.nii.gz', 'patient005_4d.nii.gz', 'patient005_frame01.nii.gz', 'patient005_frame13_gt.nii.gz', 'Info.cfg', 'patient005_frame01_gt.nii.gz']
patient005_frame13_gt.nii.gz
(256, 256, 18)
['patient006_frame01.nii.gz', 'patient006_frame16.nii.gz', 'patient006_frame16_gt.nii.gz', 'patient006_frame01_gt.nii.gz', 'patient006_4d.nii.gz', 'Info.cfg']
patient006_frame16_gt.nii.gz


In [8]:
new_data

{'x': [array([[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         ...,
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
    

In [13]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Assuming 'x' is your data and 'y' is your labels
# Data preprocessing (normalize each channel)
x = np.array(new_data['x'])
x_mean = np.mean(x, axis=(0, 1, 2))
x_std = np.std(x, axis=(0, 1, 2))
x = (x - x_mean) / x_std

y = new_data['y']

# Convert labels to integer encoding
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

# Reduce model complexity (fewer filters in Conv2D and smaller dense layer)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 18)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(set(y)), activation='softmax')
])

# Learning rate scheduling (adjust parameters)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-5,  # Reduce initial learning rate
    decay_steps=1000,
    decay_rate=0.9
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Compile the model
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Data augmentation with increased options
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(x_train)

# Increase batch size and reduce epochs
model.fit(datagen.flow(x_train, y_train, batch_size=64),
          epochs=50,  # Reduce the number of epochs
          validation_data=(x_test, y_test),
          callbacks=[early_stopping])

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:2092: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (90, 256, 256, 18) (18 channels).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (90, 256, 256, 18) (18 channels).
  warnings.warn(


Epoch 1/50
2/2 [==============================] - 14s 8s/step - loss: 3.8950 - accuracy: 0.1556 - val_loss: 2.1440 - val_accuracy: 0.1000
Epoch 2/50
2/2 [==============================] - 8s 5s/step - loss: 2.9683 - accuracy: 0.2556 - val_loss: 2.2002 - val_accuracy: 0.1000
Epoch 3/50
2/2 [==============================] - 8s 2s/step - loss: 3.3423 - accuracy: 0.2222 - val_loss: 2.2754 - val_accuracy: 0.1000
Epoch 4/50
2/2 [==============================] - 9s 6s/step - loss: 3.1334 - accuracy: 0.2556 - val_loss: 2.3532 - val_accuracy: 0.1000
Epoch 5/50
2/2 [==============================] - 9s 4s/step - loss: 2.5097 - accuracy: 0.3444 - val_loss: 2.4035 - val_accuracy: 0.1000
Epoch 6/50
2/2 [==============================] - 8s 6s/step - loss: 2.6194 - accuracy: 0.3111 - val_loss: 2.4328 - val_accuracy: 0.1000
Epoch 7/50
2/2 [==============================] - 7s 5s/step - loss: 2.5658 - accuracy: 0.2778 - val_loss: 2.4376 - val_accuracy: 0.1000
Epoch 8/50
2/2 [========================